In [1]:
#Import modules
import pandas 
import os
import pipeline_functions
from importlib import reload # reload 
reload(pipeline_functions)

<module 'pipeline_functions' from 'C:\\Users\\Mercedes_IIB-UNSAM\\Desktop\\2021\\pipeline_functions.py'>

In [3]:
#Data import
path = 'Path'
os.chdir(path)
in_file = 'gene_drug_interaction'
gdi = pandas.read_csv(in_file,sep='\t')

#Data for genes
filtro_genes_1 = pipeline_functions.leer_tabla_columnas("yeast.genes.list",'\t',['species_code', 'gene_name', 'omcl'])
filtro_genes_2 = pipeline_functions.leer_tabla_columnas("tryp.genes.list",'\t',['species_code', 'gene_name', 'omcl'])
filtro_genes_3 = pipeline_functions.leer_tabla_columnas('tbr.essentialOrthologs.list',',')
                                     
#Data for compounds
gdi_inchikey = pipeline_functions.smiles_to_inchikey(gdi,'smiles')                               
filtro_compuestos_1 = pipeline_functions.leer_tabla_columnas('tested_drugs_inchikey.csv',',')
filtro_compuestos_2 = pipeline_functions.leer_tabla_columnas('drug_availability_inchikey.csv',',')

gdi_inchikey = gdi_inchikey[['orf','inchiKey']]

# Genes filtering

In [4]:
# Assign OMCL to S. cerevisiae genes
resultado_filtro_genes_1 = pipeline_functions.interseccion_datos(gdi_inchikey,filtro_genes_1,'orf','gene_name',"OMCL in Yeast",'orf')
resultado_genes_1 = resultado_filtro_genes_1[0]
resultado_genes_1 = resultado_genes_1[['gene_name','omcl','inchiKey']]
resultado_genes_1 = resultado_genes_1.drop_duplicates()
resultado_genes_1 = resultado_genes_1.dropna()

In [5]:
# Select T. cruzi genes with shared OMCL
resultado_filtro_genes_2 = pipeline_functions.interseccion_datos(resultado_genes_1 ,filtro_genes_2,'omcl','omcl',"OMCL in Tryps", "omcl")
resultado_genes_2 = resultado_filtro_genes_2[0]
resultado_genes_2 = resultado_genes_2[['gene_name_y','omcl','inchiKey']]
resultado_genes_2 = resultado_genes_2.drop_duplicates()
resultado_genes_2 = resultado_genes_2.dropna()

In [6]:
# Select Essential genes
resultado_filtro_genes_3 = pipeline_functions.interseccion_datos(resultado_genes_2,filtro_genes_3,'omcl','omcl',"Essentiality", "omcl")
resultado_genes_3 = resultado_filtro_genes_2[0]

In [7]:
# Set promiscuity threshold
umbral=200
genes_count = resultado_filtro_genes_3[0].groupby('gene_name_y').nunique()
genes_count = genes_count.sort_values(by=['inchiKey'])
genes_count = genes_count[genes_count['inchiKey']<umbral]
genes_count = genes_count.unstack(level=-1).reset_index().drop('level_0', axis=1)
genes_count = genes_count.drop_duplicates()
gene_selection = pandas.merge(left=gdi_inchikey,right=genes_count,how="inner",left_on=['orf'], right_on=['gene_name_y'])
gene_selection = gene_selection[['orf','inchiKey']]

In [8]:
#See results
genes_count

,gene_name_y,0
0,YDR424C,1
1,YPL217C,1
2,YGR037C,1
3,YDR447C,1
4,YKL172W,1
...,...,...
1987,YKL203C,181
1988,YPL273W,190
1989,YOR265W,197
1990,YPL240C,198


In [9]:
#Create a df with deleted data
not_selected_genes = pandas.concat([resultado_filtro_genes_1[1], resultado_filtro_genes_2[1], resultado_filtro_genes_3[1]], ignore_index=False)

# Analysis of genes filter

In [10]:
resultados_1 = pipeline_functions.interseccion_datos(gdi_inchikey,resultado_genes_1,'orf','gene_name','Merged data','orf')

In [11]:
resultados_1 = resultados_1[0]
resultados_1 = resultados_1[['gene_name','omcl','inchiKey_y']]
resultados_1 = resultados_1.drop_duplicates()
resultados_1 = resultados_1.dropna()

In [12]:
resultados_1
print(len(resultados_1['gene_name'].unique()))
print(len(resultados_1['inchiKey_y'].unique()))

5008
2761


In [13]:
resultados_2 = pipeline_functions.interseccion_datos(gdi_inchikey,resultado_genes_2,'orf','gene_name_y','Merged data','orf')

In [14]:
resultados_2.head(2)


(         gene_name_y        omcl                   inchiKey_x      orf  \
 0            YGL084C  OG5_128109  GHASVSINZRGABV-UHFFFAOYSA-N  YGL084C   
 1            YGL084C  OG5_128109  GHASVSINZRGABV-UHFFFAOYSA-N  YGL084C   
 2            YGL084C  OG5_128109  GHASVSINZRGABV-UHFFFAOYSA-N  YGL084C   
 3            YGL084C  OG5_128109  GHASVSINZRGABV-UHFFFAOYSA-N  YGL084C   
 4            YGL084C  OG5_128109  GHASVSINZRGABV-UHFFFAOYSA-N  YGL084C   
 ...              ...         ...                          ...      ...   
 12538039     YPR058W  OG5_129197  JUVIOZPCNVVQFO-NDXORKPFSA-N  YPR058W   
 12538040     YPR058W  OG5_129197  JUVIOZPCNVVQFO-NDXORKPFSA-N  YPR058W   
 12538041     YPR058W  OG5_129197  JUVIOZPCNVVQFO-NDXORKPFSA-N  YPR058W   
 12538042     YPR058W  OG5_129197  JUVIOZPCNVVQFO-NDXORKPFSA-N  YPR058W   
 12538043     YPR058W  OG5_129197  JUVIOZPCNVVQFO-NDXORKPFSA-N  YPR058W   
 
                            inchiKey_y  
 0         GHASVSINZRGABV-UHFFFAOYSA-N  
 1         XUKUU

In [15]:
resultados_2 = resultados_2[0]
resultados_2 = resultados_2[['gene_name_y','omcl','inchiKey_y']]
resultados_2 = resultados_2.drop_duplicates()
resultados_2 = resultados_2.dropna()

In [16]:
resultados_2
print(len(resultados_2['gene_name_y'].unique()))
print(len(resultados_2['inchiKey_y'].unique()))

1780
2405


In [17]:
resultados_3 = pipeline_functions.interseccion_datos(gdi_inchikey,resultado_genes_3,'orf','gene_name_y','Merged data','orf')

In [18]:
resultados_3 = resultados_3[0]
resultados_3 = resultados_3[['gene_name_y','omcl','inchiKey_y']]
resultados_3 = resultados_3.drop_duplicates()
resultados_3 = resultados_3.dropna()

In [19]:
resultados_3
print(len(resultados_3['gene_name_y'].unique()))
print(len(resultados_3['inchiKey_y'].unique()))

1780
2405


In [20]:
resultados_4 = pipeline_functions.interseccion_datos(gdi_inchikey,gene_selection,'orf','orf','Merged data','orf')

In [21]:
resultados_4 = resultados_4[0]
resultados_4 = resultados_4[['orf','inchiKey_y']]
resultados_4 = resultados_4.drop_duplicates()
resultados_4 = resultados_4.dropna()

In [22]:
print(len(resultados_4))
print(len(resultados_4['orf'].unique()))
print(len(resultados_4['inchiKey_y'].unique()))

39631
996
1326


In [23]:
resultados_4

,orf,inchiKey_y
0,YOR317W,FBOZXECLQNJBKD-UHFFFAOYSA-N
1,YOR317W,XRECTZIEBJDKEO-UHFFFAOYSA-N
2,YOR317W,XTPOJZQDRYWGAK-UHFFFAOYSA-P
4,YOR317W,ZPFXAOWNKLFJDN-UHFFFAOYSA-N
5,YOR317W,RYHCACJBKCOBTJ-UHFFFAOYSA-N
...,...,...
7755568,YNL067W,FNPHNLNTJNMAEE-UHFFFAOYSA-O
7755569,YNL067W,CWJSHJJYOPWUGX-UHFFFAOYSA-N
7755570,YNL067W,IZUPBVBPLAPZRR-UHFFFAOYSA-N
7755571,YNL067W,VHKXXVVRRDYCIK-UHFFFAOYSA-N


In [24]:
results = pandas.read_csv('missing_list.csv')
filtro_genes_1 = pandas.read_csv("yeast.genes.list",sep='\t')
filtro_genes_2 = pandas.read_csv("tryp.genes.list",sep='\t')
filtro_genes_3 = pipeline_functions.leer_tabla_columnas('tbr.essentialOrthologs.list',',')             

In [25]:
df1 = pandas.merge(results, filtro_genes_1,'inner', left_on='orf',right_on='gene_name')
df2 = pandas.merge(df1, filtro_genes_2,'inner', left_on='omcl_x',right_on='omcl')
df3a = df2.loc[(df2['species_code_y'] == 'tcr')]
df3b = df2.loc[(df2['species_code_y'] == 'tcr.dm28')]
df3 = pandas.concat([df3a,df3b],sort = False)
df3.to_csv('genes_info.csv')

# Compounds filtering

In [26]:
# Select commercially available compounds
resultado_filtro_compuestos_1 = pipeline_functions.interseccion_datos(gdi_inchikey,filtro_compuestos_2,'inchiKey','inchikey','Disponibilidad comercial','inchiKey')
resultado_compuestos_1 = resultado_filtro_compuestos_1[0]
resultado_compuestos_1 = resultado_compuestos_1[['orf','inchiKey','smiles']]
resultado_compuestos_1 = resultado_compuestos_1.drop_duplicates()
resultado_compuestos_1 = resultado_compuestos_1.dropna()

In [27]:
# Select drug like and lead like compounds
resultado_filtro_compuestos_2 = pipeline_functions.drug_likness(resultado_compuestos_1,'smiles')
resultado_compuestos_2 = resultado_filtro_compuestos_2[0]
resultado_compuestos_2 = resultado_compuestos_2[['orf','inchiKey','smiles']]
resultado_compuestos_2 = resultado_compuestos_2.drop_duplicates()
resultado_compuestos_2 = resultado_compuestos_2.dropna()

In [28]:
# Set promiscuity threshold
umbral=20
drug_count = resultado_filtro_compuestos_2[0].groupby('inchiKey').nunique()
drug_count = drug_count.sort_values(by=['orf'])
drug_count = drug_count[drug_count['orf']<umbral]
drug_count = drug_count.unstack(level=-1).reset_index().drop('level_0', axis=1)
drug_count = drug_count.drop_duplicates()

drug_selection = pandas.merge(left=gdi_inchikey,right=drug_count,how="inner",left_on=['inchiKey'], right_on=['inchiKey'])

In [29]:
#Create a df with deleted data
not_selected_compounds = pandas.concat([resultado_filtro_compuestos_1[1], resultado_filtro_compuestos_2[1]], ignore_index=False)

# Analysis of compounds filter

In [30]:
resultados_1 = pipeline_functions.interseccion_datos(gdi_inchikey,resultado_compuestos_1,'inchiKey','inchiKey','Merged data','inchiKey')

In [31]:
resultados_1 = resultados_1[0]
resultados_1 = resultados_1[['orf_y','inchiKey']]
resultados_1 = resultados_1.drop_duplicates()
resultados_1 = resultados_1.dropna()

In [32]:
resultados_1
print(len(resultados_1['orf_y'].unique()))
print(len(resultados_1['inchiKey'].unique()))

5220
1195


In [33]:
resultados_2 = pipeline_functions.interseccion_datos(gdi_inchikey,resultado_compuestos_2,'inchiKey','inchiKey','Merged data','inchiKey')

In [34]:
resultados_2 = resultados_2[0]
resultados_2 = resultados_2[['orf_y','inchiKey']]
resultados_2 = resultados_2.drop_duplicates()
resultados_2 = resultados_2.dropna()

In [35]:
resultados_2
print(len(resultados_2['orf_y'].unique()))
print(len(resultados_2['inchiKey'].unique()))

5115
1126


In [36]:
resultados_3 = pipeline_functions.interseccion_datos(gdi_inchikey,drug_selection,'inchiKey','inchiKey','Merged data','inchiKey')

In [37]:
resultados_3 = resultados_3[0]
resultados_3 = resultados_3[['orf_y','inchiKey']]
resultados_3 = resultados_3.drop_duplicates()
resultados_3 = resultados_3.dropna()

In [38]:
resultados_3
print(len(resultados_3['orf_y'].unique()))
print(len(resultados_3['inchiKey'].unique()))

310
745


In [39]:
#gdi = pandas.read_csv(in_file,sep='\t')
df4 = pandas.merge(gdi, df3,'inner', left_on='inchiKey',right_on='inchiKey')
#df4 = pandas.merge(results, df3,'inner', left_on='inchiKey',right_on='inchiKey')
#results
df4.to_csv('cmp_info.csv')

# Data merge

In [40]:
resultados = pipeline_functions.interseccion_datos(drug_selection,gene_selection,'inchiKey','inchiKey','Merged data','inchiKey')

In [41]:
resultados_selección = resultados[0]
resultados_selección = resultados_selección[['orf_x','inchiKey']]
resultados_selección = resultados_selección .rename(columns={'orf_x':'orf'}) 
resultados_selección = resultados_selección.drop_duplicates()
resultados_selección = resultados_selección.dropna()

In [42]:
resultados_selección.to_csv('gdi_results.csv')

In [43]:
not_selected_data = pandas.concat([resultados[1], not_selected_genes, not_selected_compounds], ignore_index=False)

In [44]:
not_selected_data = not_selected_data[['inchiKey','orf','Filter Name']]

In [45]:
not_selected_data.to_csv('gdi_not_selected_data.csv')